# **multimoda-rs**: Tutorial examples IVUS/OCT to centerline
## Alining from files
The first example will focus on solving the problem of aligning frames within a pullback of either IVUS or OCT images.
We will start with gated IVUS images (systole/diastole) during two different states (e.g. rest/stress).
The .csv files are expected to set up in the following style:
```text
--------------------------------------------------------------------
|      185     |       5.32     |      2.37       |        0.0     |
|      185     |       5.12     |      2.46       |        0.0     |
|      ...     |       ...      |      ...        |        ...     |
```
where the first column is the frame index the point is from, the second to forth are x-, y- and z-coordinates. The naming conventions of the files are diastolic_contours.csv, diastolic_reference_points.csv, ... (see ./data). This is in alignment with the output of the [AIVUS-CAA software](https://https://github.com/AI-in-Cardiovascular-Medicine/AIVUS-CAA).

The first goal is to align the frames within a pullback by translating their centroids to a line and rotating them towards each other minimizing Hausdorff distance of the contours and catheter contours created from the image center. The influence of the catheter (which represents the image center) on the rotation can be adjusted by the number of points passed to catheter. If no catheter should be created just pass n_points=0.

In the same function states are aligned with each other (e.g. systole to diastole) and z-distance are averaged over the two states to have comparable frame positions. If heartrate is very different (e.g. rest to stress) a resampling is performed of the lower heartrate geometry.

Load packages multimodars, and for linking the numpy package.

In [ ]:
import os
import multimodars as mm
import numpy as np

# load the provided example data
os.chdir(Path.cwd().parent / "data")

# mode full and it's meaning
rest, stress, dia, sys = mm.from_file(
    mode="full", 
    rest_input_path="ivus_rest", 
    stress_input_path="ivus_stress", 
    steps_best_rotation=300, 
    range_rotation_rad=1.57, 
    rest_output_path="output/rest", 
    stress_output_path="output/stress", 
    diastole_output_path="output/diastole", 
    systole_output_path="output/systole", 
    interpolation_steps=28, 
    image_center=(4.5, 4.5),
    radius=0.5,
    n_points=20,
)

Generating geometry for "ivus_rest"
file/path                                          loaded
ivus_rest/diastolic_contours.csv                   true
ivus_rest/diastolic_reference_points.csv           true
ivus_rest/combined_sorted_manual.csv               true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_stress"
file/path                                          loaded
ivus_stress/diastolic_contours.csv                 true
ivus_stress/diastolic_reference_points.csv         true
ivus_stress/combined_sorted_manual.csv             true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_rest"
file/path                                          loaded
ivus_rest/systolic_contours.csv                    true
ivus_rest/systolic_reference_points.csv            true
ivus_rest/combined_sorted_manual.csv               true
geometry pair: systolic geometry generated
Generating geometry for "ivus_stress"
file/path                                  

In [24]:
print(rest)
print("\n")
print(rest.dia_geom)
print("\n")
print(rest.dia_geom.contours[0])

Diastolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=385, p=0, x=3.57, y=3.67, z=24.54, aortic=false) 
Systolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=319, p=0, x=3.53, y=3.46, z=20.80, aortic=false)


Geometry(17 contours, 17 walls), Catheter(17 catheter), Reference Point: Point(f=385, p=0, x=3.57, y=3.67, z=24.54, aortic=false)


Contour(id=3, points=501, centroid=(3.72, 5.25, 3.86))


The data is now neatly ordered in pairs (e.g. diastolic and systolic geometry). Every geometry has contours for lumen and walls and a created catheter. The reference point will be used to align the geometry to the centerline. All points corresponding to a contour are also save in a contour struct.

The four pairs represent all 4 possible comparison in gated images, as for example in coronary artery anomalies (rest pulsatile lumen deformation, stress pulsatile lumen deformation, stress-induced diastolic lumen deformation and stress-induced systolic lumen deformation).

For a simple pre- and post-stenting comparison do the following:

In [26]:
stent_comparison = mm.from_file(
    mode="doublepair",
    rest_input_path="ivus_prestent",
    stress_input_path="ivus_poststent",
    steps_best_rotation=300,
    range_rotation_rad=1.57,
    interpolation_steps=0
    )

Generating geometry for "ivus_prestent"
file/path                                          loaded
ivus_prestent/diastolic_contours.csv               true
ivus_prestent/diastolic_reference_points.csv       true
ivus_prestent/combined_sorted_manual.csv           true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_poststent"
file/path                                          loaded
ivus_poststent/diastolic_contours.csv              true
ivus_poststent/diastolic_reference_points.csv      true
ivus_poststent/combined_sorted_manual.csv          true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_prestent"
file/path                                          loaded
ivus_prestent/systolic_contours.csv                true
ivus_prestent/systolic_reference_points.csv        true
ivus_prestent/combined_sorted_manual.csv           true
geometry pair: systolic geometry generated
Generating geometry for "ivus_poststent"
file/path                    

## Alignment from array
While the alignment from file is one option, the more flexible option is to create Geometries directly from numpy array, and then perform the same operations with these Geometries.

In [1]:
# prepare data from files
contours_pre = np.genfromtxt("ivus_prestent/diastolic_contours.csv", delimiter='\t', names=["id", "x", "y", "z"])
points = [mm.PyContourPoint(idx.astype(int), i, x, y, z, False) for i, (idx, x, y, z) in enumerate(contours_pre)]
catheter_pre = mm.create_catheter_contours(points, image_center=(4.5,4.5)).to_array()
print(catheter_pre)
ref_pre = np.genfromtxt("ivus_prestent/diastolic_reference_points.csv", delimiter='\t')
contours_post = np.genfromtxt("ivus_poststent/diastolic_contours.csv", delimiter='\t')
ref_post = np.genfromtxt("ivus_poststent/diastolic_reference_points.csv", delimiter='\t')

geom_pre = mm.numpy_to_geometry_layers(contours_arr=contours_pre, catheter_arr=catheter_pre, walls_arr=np.ndarray([]), reference_arr=ref_pre)
geom_post = mm.numpy_to_geometry_layers(contours_arr=contours_post, reference_arr=ref_post)

NameError: name 'np' is not defined